In [1]:
import operator
import os
import warnings
from dataclasses import dataclass
from random import randint, choices
from typing import Callable, Iterable, Tuple

import matplotlib
import numpy as np
import sympy as sp
from numpy.random import choice
from sympy import latex
from sympy.abc import x, y

In [2]:
warnings.filterwarnings("error")
matplotlib.use("Agg")
dx, dy = sp.symbols('dx dy')

In [3]:
%run document_generator.ipynb

In [4]:
def latex_image(path: str) -> str:
    return r"""\begin{figure}[H]
    \centering
    \includegraphics[]{%s}
    \label{fig:figure}
\end{figure}""" % path

In [5]:
def random_float_coef():
    return sp.S(choice(list(set(range(-3, 4)) - {0}))) / choice([1, 2, 3])


def generate_combination_x_y() -> sp.Expr:
    return choices([sum([random_float_coef() * item for item in choice([x ** 2, y ** 2, x * y, x, y, 1], size=3)]),
                    sum([random_float_coef() * item for item in
                         choice([x ** 2, y ** 2, x * y, x, y, 1], size=2, replace=False)]),
                    random_float_coef() * x * y
                    ], weights=[3, 4, 3])[0]


def func_generator_decorator(expr: sp.Expr) -> Callable[[], sp.Expr]:
    def generate_func() -> sp.Expr:
        symbol = list(expr.free_symbols)[0]
        return random_float_coef() * expr.subs(symbol, random_float_coef() * symbol)

    return generate_func


generate_asin = func_generator_decorator(sp.asin(x))
generate_acos = func_generator_decorator(sp.acos(x))
generate_atan = func_generator_decorator(sp.atan(x))
generate_sin = func_generator_decorator(sp.sin(x))
generate_cos = func_generator_decorator(sp.cos(x))
generate_tan = func_generator_decorator(sp.tan(x))


def generate_raising_to_negative_int() -> sp.Expr:
    return random_float_coef() * sp.S(1) / x


def generate_linear_func() -> sp.Expr:
    return random_float_coef() * x + random_float_coef()


def generate_id() -> sp.Expr:
    return x


def generate_exponentiation() -> sp.Expr:
    coef1 = random_float_coef()
    degree_basis = 1
    while degree_basis == 1:
        degree_basis = choice([choice([2, 3, 5]), sp.E])
    return coef1 * degree_basis ** x


def generate_root() -> sp.Expr:
    return random_float_coef() * choice([sp.sqrt, sp.cbrt])(random_float_coef() * x)


def composition(first, second):
    return first.subs(x, second)


def generate_random_func():
    generators = [generate_asin,
                  generate_acos,
                  generate_atan,
                  generate_sin,
                  generate_cos,
                  generate_tan,
                  generate_linear_func,
                  generate_id,
                  generate_root]
    operators = [operator.add, operator.mul, operator.truediv]
    func = None
    while func is None or len(latex(func)) > 50 or len(func.free_symbols) < 2:
        func1 = choice(generators)().subs(x, generate_combination_x_y())
        func2 = choice(generators)().subs(x, generate_combination_x_y())
        random_operator = choice(operators)
        func = random_operator(func1, func2)
    return func

In [6]:
DESCRIPTION = "Построить линии уровня и вычислить полный дифференциал функции f(x, y)."

OUT_ANSWER_PATH = "../out/answers/answer14/"
os.makedirs(os.path.dirname(OUT_ANSWER_PATH), exist_ok=True)

In [7]:
@dataclass
class Color:
    human_version: str
    plot_version: str


@dataclass
class LevelLine:
    level: sp.Number
    color: Color

In [8]:
def task_condition(expr: sp.Expr, level_lines: Iterable[LevelLine]) -> str:
    level_lines_task = ', '.join(map(
        lambda level_line: f'${latex(level_line.level)}$ - {level_line.color.human_version} цветом',
        level_lines)
    )
    return f"\n\n$f(x, y) = {latex(expr)}$\n\n" \
           f"Построить линии уровня для значений: {level_lines_task}"

In [9]:
def solve_task(option: int, expr: sp.Expr, level_lines: Iterable[LevelLine]) -> str:
    full_diff = (expr.diff(x) * dx + expr.diff(y) * dy).simplify()
    p = sp.plot(ylabel='y', xlim=(-5, 5), ylim=(-5, 5), show=False)
    for level_line in level_lines:
        p.append(sp.plot_implicit(
            sp.Eq(expr, level_line.level), (x, -5, 5), (y, -5, 5),
            line_color=level_line.color.plot_version,
            show=False)[0])
    p.save(OUT_ANSWER_PATH + str(option))
    matplotlib.pyplot.close()
    return f"\n\n{latex_image(OUT_ANSWER_PATH + str(option) + '.png')}\n\n" \
           f"Полный дифференциал функции ${latex(full_diff)}$"

In [10]:
def generate_random_levels(expr: sp.Expr, count: int) -> list[sp.Number]:
    with np.errstate(all="ignore"):
        np_func = sp.lambdify((x, y), expr, 'numpy')
        values = np_func(*np.meshgrid(np.linspace(-5, 5), np.linspace(-5, 5)))
        min_val = int(np.nanmin(values)) + 1
        max_val = int(np.nanmax(values)) - 1
        if min_val > max_val or max_val - min_val < 6:
            raise UserWarning
        levels = []
        while len(levels) < count:
            levels.append(randint(min_val, max_val))
        return levels


def generate_task_and_answer(option: int) -> Tuple[str, str]:
    colors = [Color("зеленым", "green"),
              Color("красным", "red"),
              Color("синим", "blue"),
              Color("желтым", "yellow"),
              Color("фиолетовым", "magenta"),
              Color("черным", "black")]
    while True:
        func = None
        try:
            func = generate_random_func()
            level_line_count = randint(3, 5)
            levels = generate_random_levels(func, level_line_count)
            level_lines = list(map(lambda pair: LevelLine(pair[0], pair[1]),
                                   zip(levels, choice(colors, size=level_line_count, replace=False)))
                               )
            return task_condition(func, level_lines), solve_task(option, func, level_lines)
        except UserWarning:
            pass
        except Exception as e:
            print(e)
            display(func)

In [11]:
doc = DocumentGenerator(generate_task_and_answer, DESCRIPTION)

In [12]:
write_tasks_and_solutions(doc, "../out/tasks/task-14.tex", OUT_ANSWER_PATH + "answer-14.tex", 150)